In [25]:
import pandas as pd
import numpy as np
from pyparsing import col

In [4]:
cols = {0: 'pseudo', 1: 'code'}

train_df = pd.read_csv('../data/input-tok-train-shuf.tsv', header=None, delimiter='\t')
train_df.rename(columns=cols, inplace=True)

train_df['pseudo_token'] = train_df['pseudo'].str.split(' ')
train_df['code_token'] = train_df['code'].str.split(' ')
train_df.head()

,pseudo,code,pseudo_token,code_token
0,set l to mid,l = mid ;,"[set, l, to, mid]","[l, =, mid, ;]"
1,if i is 0,if ( i == 0 ),"[if, i, is, 0]","[if, (, i, ==, 0, )]"
2,read n and k,cin >> n >> k ;,"[read, n, and, k]","[cin, >>, n, >>, k, ;]"
3,declare long longs sum = 0 and min = LONG_LONG...,"long long min = LONG_LONG_MAX , sum = 0 ;","[declare, long, longs, sum, =, 0, and, min, =,...","[long, long, min, =, LONG_LONG_MAX, ,, sum, =,..."
4,dy = integer array where the the following int...,"int dy [ ] = { 0 , 0 , - 1 , 1 } ;","[dy, =, integer, array, where, the, the, follo...","[int, dy, [, ], =, {, 0, ,, 0, ,, -, 1, ,, 1, ..."


In [18]:
def create_binary_seq_from_row(row):
    """
    Returns binary sequence for pseudocode tokens based on 
    true code tokens

    If the pseudocode token exists in the true code (can be
    copied), then the sequence contains a 1 in that position
    """
    code_token_set = set(row['code_token'])
    output_seq = []

    for token in row['pseudo_token']:
        if token in code_token_set:
            output_seq.append(1)
        else:
            output_seq.append(0)

    assert len(output_seq) == len(row['pseudo_token'])
    return output_seq

In [19]:
code_binary_seq = train_df.apply(create_binary_seq_from_row, axis=1)
train_df['code_binary_seq'] = code_binary_seq
train_df

,pseudo,code,pseudo_token,code_token,code_binary_seq
0,set l to mid,l = mid ;,"[set, l, to, mid]","[l, =, mid, ;]","[0, 1, 0, 1]"
1,if i is 0,if ( i == 0 ),"[if, i, is, 0]","[if, (, i, ==, 0, )]","[1, 1, 0, 1]"
2,read n and k,cin >> n >> k ;,"[read, n, and, k]","[cin, >>, n, >>, k, ;]","[0, 1, 0, 1]"
3,declare long longs sum = 0 and min = LONG_LONG...,"long long min = LONG_LONG_MAX , sum = 0 ;","[declare, long, longs, sum, =, 0, and, min, =,...","[long, long, min, =, LONG_LONG_MAX, ,, sum, =,...","[0, 1, 0, 1, 1, 1, 0, 1, 1, 1]"
4,dy = integer array where the the following int...,"int dy [ ] = { 0 , 0 , - 1 , 1 } ;","[dy, =, integer, array, where, the, the, follo...","[int, dy, [, ], =, {, 0, ,, 0, ,, -, 1, ,, 1, ...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
...,...,...,...,...,...
181857,declare static constant integer mod = 1000000009,static const int mod = 1000000009 ;,"[declare, static, constant, integer, mod, =, 1...","[static, const, int, mod, =, 1000000009, ;]","[0, 1, 0, 0, 1, 1, 1]"
181858,print NO and a new line,"cout << "" NO "" << ' \n ' ;","[print, NO, and, a, new, line]","[cout, <<, "", NO, "", <<, ', \n, ', ;]","[0, 1, 0, 0, 0, 0]"
181859,change the value of ans to abs ( x - y ) / d,ans = abs ( x - y ) / d ;,"[change, the, value, of, ans, to, abs, (, x, -...","[ans, =, abs, (, x, -, y, ), /, d, ;]","[0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]"
181860,else if s is less than f,else if ( s < f ),"[else, if, s, is, less, than, f]","[else, if, (, s, <, f, )]","[1, 1, 1, 0, 0, 0, 1]"


In [20]:
def create_tagged_tuples(row):
    """
    Zips pseudo_token and code_binary_seq
    """
    z = zip(row['pseudo_token'], row['code_binary_seq'])
    return list(z)

In [21]:
train_df['tagged_pseudo'] = train_df.apply(create_tagged_tuples, axis=1)
train_df

,pseudo,code,pseudo_token,code_token,code_binary_seq,tagged_pseudo
0,set l to mid,l = mid ;,"[set, l, to, mid]","[l, =, mid, ;]","[0, 1, 0, 1]","[(set, 0), (l, 1), (to, 0), (mid, 1)]"
1,if i is 0,if ( i == 0 ),"[if, i, is, 0]","[if, (, i, ==, 0, )]","[1, 1, 0, 1]","[(if, 1), (i, 1), (is, 0), (0, 1)]"
2,read n and k,cin >> n >> k ;,"[read, n, and, k]","[cin, >>, n, >>, k, ;]","[0, 1, 0, 1]","[(read, 0), (n, 1), (and, 0), (k, 1)]"
3,declare long longs sum = 0 and min = LONG_LONG...,"long long min = LONG_LONG_MAX , sum = 0 ;","[declare, long, longs, sum, =, 0, and, min, =,...","[long, long, min, =, LONG_LONG_MAX, ,, sum, =,...","[0, 1, 0, 1, 1, 1, 0, 1, 1, 1]","[(declare, 0), (long, 1), (longs, 0), (sum, 1)..."
4,dy = integer array where the the following int...,"int dy [ ] = { 0 , 0 , - 1 , 1 } ;","[dy, =, integer, array, where, the, the, follo...","[int, dy, [, ], =, {, 0, ,, 0, ,, -, 1, ,, 1, ...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[(dy, 1), (=, 1), (integer, 0), (array, 0), (w..."
...,...,...,...,...,...,...
181857,declare static constant integer mod = 1000000009,static const int mod = 1000000009 ;,"[declare, static, constant, integer, mod, =, 1...","[static, const, int, mod, =, 1000000009, ;]","[0, 1, 0, 0, 1, 1, 1]","[(declare, 0), (static, 1), (constant, 0), (in..."
181858,print NO and a new line,"cout << "" NO "" << ' \n ' ;","[print, NO, and, a, new, line]","[cout, <<, "", NO, "", <<, ', \n, ', ;]","[0, 1, 0, 0, 0, 0]","[(print, 0), (NO, 1), (and, 0), (a, 0), (new, ..."
181859,change the value of ans to abs ( x - y ) / d,ans = abs ( x - y ) / d ;,"[change, the, value, of, ans, to, abs, (, x, -...","[ans, =, abs, (, x, -, y, ), /, d, ;]","[0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]","[(change, 0), (the, 0), (value, 0), (of, 0), (..."
181860,else if s is less than f,else if ( s < f ),"[else, if, s, is, less, than, f]","[else, if, (, s, <, f, )]","[1, 1, 1, 0, 0, 0, 1]","[(else, 1), (if, 1), (s, 1), (is, 0), (less, 0..."


In [22]:
tags = {0, 1}

tagged_pseudo_list = [tup for row in train_df['tagged_pseudo'] for tup in row]
tagged_pseudo_list

[('set', 0),
 ('l', 1),
 ('to', 0),
 ('mid', 1),
 ('if', 1),
 ('i', 1),
 ('is', 0),
 ('0', 1),
 ('read', 0),
 ('n', 1),
 ('and', 0),
 ('k', 1),
 ('declare', 0),
 ('long', 1),
 ('longs', 0),
 ('sum', 1),
 ('=', 1),
 ('0', 1),
 ('and', 0),
 ('min', 1),
 ('=', 1),
 ('LONG_LONG_MAX', 1),
 ('dy', 1),
 ('=', 1),
 ('integer', 0),
 ('array', 0),
 ('where', 0),
 ('the', 0),
 ('the', 0),
 ('following', 0),
 ('integers', 0),
 ('are', 0),
 ('initialized', 0),
 ('starting', 0),
 ('at', 0),
 ('dy', 1),
 ('[', 1),
 ('0', 1),
 (']', 1),
 (':', 0),
 ('0', 1),
 (',', 1),
 ('0', 1),
 (',', 1),
 ('-', 1),
 ('1', 1),
 (',', 1),
 ('1', 1),
 ('declare', 0),
 ('integer', 0),
 ('sum', 1),
 ('=', 1),
 ('0', 1),
 ('increase', 0),
 ('ans', 1),
 ('by', 0),
 ('t', 1),
 ('for', 1),
 ('i', 1),
 ('=', 1),
 ('0', 1),
 ('to', 0),
 ('n', 1),
 ('exclusive', 0),
 ('set', 0),
 ('min', 1),
 ('to', 0),
 ('salt', 1),
 ('for', 1),
 ('i', 1),
 ('=', 1),
 ('0', 1),
 (',', 0),
 ('incrementing', 0),
 ('i', 1),
 ('if', 1),
 ('n', 1)

In [23]:
def word_given_tag(word, tag, train_bag=tagged_pseudo_list):
    """
    Compute emission probability

    Parameters:

    word: pseudocode word to check for
    tag: 0 or 1
    train_bag: corpus of pairs of tagged words
    """

    # Pairs with the tag specified
    tag_list = [pair for pair in train_bag if pair[1]==tag]

    # Number of times that tag occurred
    count_tag = len(tag_list)

    # Pairs with the word specified
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]

    # Total number of times the passed word occurred as the passed tag
    count_w_given_tag = len(w_given_tag_list)
 
    return (count_w_given_tag, count_tag)

In [24]:
def t2_given_t1(t2, t1, train_df_tagged=train_df['tagged_pseudo']):
    """
    Compute transition probability

    Parameters:

    t1: tag at time t
    t2: tag at time t+1
    train_df_tagged: column of df with tagegd pairs
    """
    
    count_t1 = 0
    count_t2_t1 = 0

    for row in train_df_tagged:

        # List of sequence of tags (hidden states)
        tags = [pair[1] for pair in row]

        # Number of t1 occurrences
        count_t1 += len([t for t in tags if t==t1])
        
        # Count of t1 followed by t2 occurrences
        for index in range(len(tags)-1):
            if tags[index] == t1 and tags[index+1] == t2:
                count_t2_t1 += 1
    
    return (count_t2_t1, count_t1)

In [33]:
# Creating t x t transition matrix of tags, t = no of tags (2)
# Matrix(i, j) represents P(jth tag after the ith tag)
 
trans_matrix = np.zeros((len(tags), len(tags)), dtype='float32')

for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        trans_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(trans_matrix)

[[0.39656797 0.53022957]
 [0.2565598  0.581478  ]]


In [34]:
# Convert the matrix to a df for better readability

trans_df = pd.DataFrame(trans_matrix, columns = list(tags), index=list(tags))
trans_df

,0,1
0,0.396568,0.530230
1,0.256560,0.581478


In [69]:
# Initial probabilities

def find_initial_probabilities(tag, train_df_tagged=train_df['tagged_pseudo']):
    """
    Compute initial probabilities

    Parameters: 
    
    tag: 
    train_df_tagged: column of df with tagegd pairs
    """

    count_tag = 0
    total_observations = train_df_tagged.size

    for row in train_df_tagged:

        if row[0][1] == tag:
            count_tag += 1
    
    return (count_tag, total_observations)
    


In [81]:
# Initial probabilities

pi_matrix = np.zeros(len(tags), dtype='float32')

for i, tag in enumerate(list(tags)):
    count_tag, total = find_initial_probabilities(tag)
    pi_matrix[i] = count_tag/total
 
# print(pi_matrix)

# Convert the matrix to a df for better readability

pi_df = pd.DataFrame(pi_matrix.T, index=list(tags))
pi_df

,0
0,0.545661
1,0.454339


In [132]:
def Viterbi(corpus_df, A_df, pi_df, B_callback):
    """
    Viterbi algorithm

    Parameters:

    corpus_df: dataframe column of word lists
    A_df: transition matrix
    pi_df: initial state
    B_callback: emission probability callback
    """
    
    states = []
    T = [0, 1]
    

    for row in corpus_df:
        state = []
        for index, word in enumerate(row):

            # Initialise list of probability column for a given observation
            p = []

            for tag in T:
                # Calculate p[0] and p[1]
                # Probability of it being 0 or 1

                # print('tag =', tag)
                # print('index =', index)
                # print('word =', word)

                if index == 0:
                    # First word of the sentence
                    # Eg: ['set', 'l', 'to', '0']
                    # index = 0, word = 'set'

                    transition_p = pi_df.loc[tag][0]
                else:
                    transition_p = A_df.loc[state[-1], tag]
                    
                # Compute emission and state probabilities
                w_given_t, t_count = B_callback(word=word, tag=tag) 
                emission_p = w_given_t/t_count
                
                state_probability = emission_p * transition_p  
                p.append(state_probability)
                
            # Getting state for which probability is maximum
            pmax, state_max = max(zip(p, T), key=lambda x: x[0])
            
            state.append(state_max)
        #     print('word:', word)
        #     print('p:', p)
        #     print('state_max:', state_max)
            
        print('current state:', state)
        states.append(state)

    return states

In [130]:
train_df['code_token'][0:1]

0    [l, =, mid, ;]
Name: code_token, dtype: object

In [133]:
Viterbi(train_df['code_token'][0:5], trans_df, pi_df, word_given_tag)

current state: [1, 1, 1, 1]
current state: [1, 1, 1, 0, 1, 1]
current state: [1, 0, 1, 0, 1, 1]
current state: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
current state: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


[[1, 1, 1, 1],
 [1, 1, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [29]:
cols = {0: 'pseudo', 1: 'code'}

test_df = pd.read_csv('../data/input-tok-test.tsv', header=None, delimiter='\t')
test_df.rename(columns=cols, inplace=True)

test_df['pseudo_token'] = train_df['pseudo'].str.split(' ')
test_df['code_token'] = train_df['code'].str.split(' ')
test_df.head()

,pseudo,code,pseudo_token,code_token
0,create integer n,int n ;,"[set, l, to, mid]","[l, =, mid, ;]"
1,read n,cin >> n ;,"[if, i, is, 0]","[if, (, i, ==, 0, )]"
2,"create integers cur , cnt with cur = 1 , cnt = 0","int cur = 1 , cnt = 0 ;","[read, n, and, k]","[cin, >>, n, >>, k, ;]"
3,create integer vector ans,vector < int > ans ;,"[declare, long, longs, sum, =, 0, and, min, =,...","[long, long, min, =, LONG_LONG_MAX, ,, sum, =,..."
4,for i = 0 to n exclusive,for ( int i = 0 ; i < n ; i ++ ),"[dy, =, integer, array, where, the, the, follo...","[int, dy, [, ], =, {, 0, ,, 0, ,, -, 1, ,, 1, ..."


In [30]:
test_df['code_binary_seq'] = test_df.apply(create_binary_seq_from_row, axis=1)
test_df

,pseudo,code,pseudo_token,code_token,code_binary_seq
0,create integer n,int n ;,"[set, l, to, mid]","[l, =, mid, ;]","[0, 1, 0, 1]"
1,read n,cin >> n ;,"[if, i, is, 0]","[if, (, i, ==, 0, )]","[1, 1, 0, 1]"
2,"create integers cur , cnt with cur = 1 , cnt = 0","int cur = 1 , cnt = 0 ;","[read, n, and, k]","[cin, >>, n, >>, k, ;]","[0, 1, 0, 1]"
3,create integer vector ans,vector < int > ans ;,"[declare, long, longs, sum, =, 0, and, min, =,...","[long, long, min, =, LONG_LONG_MAX, ,, sum, =,...","[0, 1, 0, 1, 1, 1, 0, 1, 1, 1]"
4,for i = 0 to n exclusive,for ( int i = 0 ; i < n ; i ++ ),"[dy, =, integer, array, where, the, the, follo...","[int, dy, [, ], =, {, 0, ,, 0, ,, -, 1, ,, 1, ...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
...,...,...,...,...,...
15178,if i is odd,if ( i % 2 != 0 ),"[read, n, and, k]","[cin, >>, n, >>, k, ;]","[0, 1, 0, 1]"
15179,remove first element from v,v . erase ( v . begin ( ) ) ;,"[create, integers, n, ,, k, ,, last, with, las...","[int, n, ,, k, ,, last, =, 0, ;]","[0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1]"
15180,else,else,"[increase, cnt1, by, one]","[cnt1, ++, ;]","[0, 1, 0, 0]"
15181,remove the last element from v,v . pop_back ( ) ;,"[if, x, <, =, last]","[if, (, x, <=, last, )]","[1, 1, 0, 0, 1]"


In [31]:
Viterbi(test_df['pseudo_token'], tags_df)

['set', 'l', 'to', 'mid']
['if', 'i', 'is', '0']
['read', 'n', 'and', 'k']
['declare', 'long', 'longs', 'sum', '=', '0', 'and', 'min', '=', 'LONG_LONG_MAX']
['dy', '=', 'integer', 'array', 'where', 'the', 'the', 'following', 'integers', 'are', 'initialized', 'starting', 'at', 'dy', '[', '0', ']', ':', '0', ',', '0', ',', '-', '1', ',', '1']
['declare', 'integer', 'sum', '=', '0']
['increase', 'ans', 'by', 't']
['for', 'i', '=', '0', 'to', 'n', 'exclusive']
['set', 'min', 'to', 'salt']
['for', 'i', '=', '0', ',', 'incrementing', 'i']
['if', 'n', '>', '=', '5']
['if', 'l', 'modulo', 'k']
['print', '(', '(', 'i', '+', '1', ')', '*', '(', 'i', '+', '1', ')', ')', '*', 'i', '-', '2']
['else']
['X', ',', 'Y', '=', 'sets', 'of', 'integers']
['define', 'string', 'variable', 's']
['else']
['set', 'mp', '[', "'", '7', "'", ']', 'to', '2']
['else']
['print', 'p']
['create', 'integer', 'cnt', 'with', 'cnt', '=', '0']
['let', 'val', 'be', 'a', 'integer', 'with', 'val', '=', 'n', '-', 'i']
['if', 'm

In [57]:
import torch

a = torch.rand(3,2)
a

tensor([[0.9273, 0.4056],
        [0.9737, 0.0245],
        [0.9392, 0.9372]])

In [58]:
b = torch.nn.functional.softmax(a)
b

/Users/vibhamasti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([[0.6275, 0.3725],
        [0.7210, 0.2790],
        [0.5005, 0.4995]])

In [56]:
torch.distributions.categorical.Categorical(b).sample().item()

0

In [61]:
b[:,1]

tensor([0.3725, 0.2790, 0.4995])

In [138]:
from hmmlearn.hmm import MultinomialHMM

In [139]:
h = MultinomialHMM(n_components=2)